In [25]:
import numpy as np
import qiskit as q
from qiskit import QuantumCircuit, QuantumRegister, AncillaRegister
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import Statevector
from qiskit.visualization import array_to_latex
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  
import matplotlib.cm as cm
from qiskit.circuit.library import Diagonal
import qiskit.circuit.library as qlib
%matplotlib inline
from qiskit import transpile

from qiskit_aer import AerSimulator

import time
import math

In [26]:
# for boundaries
def top_half(A):
    def convert(val):
        return val+1j*np.sqrt(1-val**2)
    func = np.vectorize(convert)
    return func(A)
def bottom_half(A):
    def convert(val):
        return val-1j*np.sqrt(1-val**2)
    func = np.vectorize(convert)
    return func(A)

In [27]:
#for collision gates
def createLambda(oper, isC1 = True):
    ctrl = oper

    # GENERALIZED
    if oper >= 8:
        oper -= 8
        
    s = bin(ctrl)[2:]
    s = '0'*int((4-len(s))) + s

    nlambda = lambdas[oper]
    
    if not isC1:
        nlambda*=-1

    qc = QuantumCircuit(2+nlinks)
    tmp = QuantumCircuit(1)

    tmp.p(nlambda, 0)
    tmp.x(0)
    tmp.p(nlambda, 0)
    tmp.x(0)
    
    gate = tmp.to_gate(label = "Lambda"+str(ctrl)).control(nlinks+1,ctrl_state = s)
    
    cbits = [i for i in range(1,nlinks+2)]
    cbits.append(0)
    qc.append(gate,cbits)
    return qc

In [28]:
#prop gates
def rshift(n):
    circ = QuantumCircuit(n)
    for i in range(n):
        if i == n-1:
            circ.x(i)
        else:
            circ.mcx(list(range(i+1,n)), i)
    return circ

def lshift(n):
    circ = QuantumCircuit(n)
    for i in reversed(range(n)):
        if i == n-1:
            circ.x(i)
        else:
            circ.mcx(list(range(i+1,n)), i)
    return circ

In [29]:
def streamCirc():
    q = QuantumRegister(nlinks+dim*nlat+1,'q')
    a = AncillaRegister(1,'a')

    setup = QuantumCircuit(q)
    setup.add_register(a)

    h1 = qlib.HGate().control(1,ctrl_state = 0)

    cb = [a,0]
    cb.extend([i for i in range(dim*nlat, dim*nlat+nlinks+1)])

    # GENERALIZED
    setup.h(a)
    # GENERALIZED
    for i in range(dirs):
        setup.append(createLambda(i,True).to_gate(label = 'Lambda'+str(i)).control(1,ctrl_state = 0),cb)

    for i in range(dirs):
        setup.append(createLambda(i,True).to_gate(label = 'Lambda'+str(i)).control(1,ctrl_state = 0),cb)
    for i in range(8,13):
        setup.append(createLambda(i,True).to_gate(label = 'Lambda'+str(i)).control(1,ctrl_state = 0),cb)
    for i in range(dirs):
        setup.append(createLambda(i,False).to_gate(label = 'Lambda'+str(i)).control(1,ctrl_state = 1),cb)
    for i in range(8,13):
        setup.append(createLambda(i,False).to_gate(label = 'Lambda'+str(i)).control(1,ctrl_state = 1),cb)
    setup.h(q[n - 2])
    setup.h(a)
    
    setup.barrier()
    L1 = lshift(nlat).to_gate(label = "L").control(nlinks+1,ctrl_state = '0001')
    R1 = rshift(nlat).to_gate(label = "R").control(nlinks+1,ctrl_state = '0010')
    L2 = lshift(nlat).to_gate(label = "L").control(nlinks+1,ctrl_state = '0011')
    R2 = rshift(nlat).to_gate(label = "R").control(nlinks+1,ctrl_state = '0100')

    # GENERALIZED
    log_M = round(math.log2(M))

    shift_controls = [n - 4, n - 3, n - 2]

    one = shift_controls + [a[0]] + [i for i in reversed(range(log_M))]
    two = shift_controls + [a[0]] + [i for i in reversed(range(log_M, 2*log_M))]
    
    setup.append(L1, one)
    setup.append(R1, one)
    setup.append(L2, two)
    setup.append(R2, two)
    setup.barrier()


    # GENERALIZED
    for i in shift_controls:
        setup.h(q[i])

    return setup


In [30]:
#calculating derivs and diagonal array
def uv(streamfunc):
    #u = streamfunction_y, v = -streamfunction_x
    v = (streamfunc[:,1:]-streamfunc[:,:-1])
    u = streamfunc[1:,:]-streamfunc[:-1,:]
    
    v = np.append(v,np.reshape(v[:,-1],(M,1)),axis=1)
    u = np.append(u,u[-1:],axis=0)
    
    v*=-1
    streamfunc = np.reshape(streamfunc,(M*M))
    return u,v

def collision_f(u_vels, v_vels):
    # for f, weights have to be calculated by advection diffusion eq, but u and v must be calculated
    def coef(link,vel):
        return w[link]*(1+e[link]*vel/cs**2)
    func = np.vectorize(coef)
    
    ret = np.zeros((5,M,M))
    # TODO: GENERALIZE
    ret[0] = func(0,np.zeros((M,M)))
    ret[1] = func(1,u_vels)
    ret[2] = func(2,u_vels)
    ret[3] = func(3,v_vels)
    ret[4] = func(4,v_vels)
    return ret.flatten()

In [31]:
def vortCirc(stream):
    q = QuantumRegister(nlinks+dim*nlat+1,'q')

    setup = QuantumCircuit(q)
    
    h1 = qlib.HGate().control(1,ctrl_state = 0)
    h3 = qlib.HGate().control(2,ctrl_state = '00')
    
    uvel,vvel = uv(stream)
    A_diag = collision_f(uvel,vvel)
    zeros = np.zeros(M*M)
    A_diag = np.concatenate((A_diag, zeros, zeros, zeros))
    B1_diag = top_half(A_diag)
    B2_diag = bottom_half(A_diag)
    
    # GENERALIZED
    setup.h(n - 1)

    Col1_diag = Diagonal(list(B1_diag))
    Col1 = Col1_diag.to_gate(label='c1')
    
    Col2_diag = Diagonal(list(B2_diag))
    Col2 = Col2_diag.to_gate(label='c2')
    
    # GENERALIZED
    indices = [n - 1] + [i for i in range(n - 1)]
    setup.append(Col1.control(1,ctrl_state = '0'), indices)
    setup.append(Col2.control(1,ctrl_state = '1'), indices)

    # GENERALIZED
    setup.h(n - 1)



    setup.barrier()
    L1 = lshift(nlat).to_gate(label = "L").control(nlinks,ctrl_state = '001')
    R1 = rshift(nlat).to_gate(label = "R").control(nlinks,ctrl_state = '010')
    L2 = lshift(nlat).to_gate(label = "L").control(nlinks,ctrl_state = '011')
    R2 = rshift(nlat).to_gate(label = "R").control(nlinks,ctrl_state = '100')
    
    # GENERALIZED
    log_M = round(math.log2(M))
    one = [n - 4, n - 3, n - 2] + [i for i in reversed(range(log_M))]
    two = [n - 4, n - 3, n - 2] + [i for i in reversed(range(log_M, 2*log_M))]
    setup.append(L1, one)
    setup.append(R1, one)
    setup.append(L2, two)
    setup.append(R2, two)
    setup.barrier()

    # GENERALIZED
    setup.h(n - 4)
    setup.h(n - 3)
    setup.h(n - 2)

    return setup

In [32]:
def calcBounds(streamfunction):
    arr = np.zeros((M,M))
    arr[-1] = -2*streamfunction[-2]
    arr[:,0] = -2*streamfunction[:,1]
    arr[:,-1] = -2*streamfunction[:,-2]
    arr[0] = -2*streamfunction[1]-2*U
    return arr

In [33]:
def Vtimestep(vort, stream):
    bounds = calcBounds(stream)
    zeros = np.zeros((M,M))
    vort = np.concatenate((vort,vort,vort,vort,vort,zeros,zeros,zeros)).flatten()
    vortSV = Statevector(vort).expand([1,0]).evolve(vortCirc(stream))
    vortAr = np.reshape(np.array(vortSV)[:M*M],(M,M))
    vortAr = np.real(vortAr)*2**(3/2)
    vortAr[0] = bounds[0]
    vortAr[-1] = bounds[-1]
    vortAr[:,0] = bounds[:,0]
    vortAr[:,-1] = bounds[:,-1]
    return np.reshape(vortAr,(M,M)), vort

def Stimestep(stream, source):
    zeros = np.zeros((M,M))
    stream = np.concatenate((stream,stream,stream,stream,stream,zeros,zeros,zeros,source,source,source,source,source,zeros,zeros,zeros)).flatten()
    streamSV = Statevector(stream).expand([1,0]).evolve(streamCirc())
    streamAr = np.reshape(np.array(streamSV)[:M*M],(M,M))
    streamAr = np.real(streamAr)*2**(4/2)
    streamAr[0] = 0
    streamAr[-1] = 0
    streamAr[:,0] = 0
    streamAr[:,-1] = 0
    return np.reshape(streamAr,(M,M)), stream

In [34]:
def normalized_statevector(SV):
    sv_array = np.array(SV)
    norm = np.linalg.norm(sv_array)
    if norm == 0:
        print("Zero!")
    sv_array = sv_array / norm
    SV_normalized = Statevector(sv_array)
    return SV_normalized

## Encoding Cost vs. Number Lattice Points

In [35]:
# select backend
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime.fake_provider import FakeBrisbane
from qiskit.quantum_info import random_statevector, Statevector
from qiskit import schedule

import pickle as pkl

service = QiskitRuntimeService()

In [ ]:
min = 5
max = 5

iter = 4

outpath = "data/twoCircuitEncodingCost5.pkl"

data = {
    "depth":[],
    "count_ops":[],
    "runtime":[]
}

for i in range(min, max + 1):
    print(i)
    for j in range(iter):
        print(f"Iteration {j}")

        M = int(math.pow(2, i)) # lattices

        dim = 2
        dirs = 5

        nlat = int(np.ceil(np.log2(M)))
        nlinks = int(np.ceil(np.log2(dirs)))

        n = nlinks+dim*nlat+1

        qc = QuantumCircuit(n)
        vec = random_statevector(int(math.pow(2, n)))
        
        qc.prepare_state(vec, list(range(n)))

        backend = FakeBrisbane()
        pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
        qc_opt = pm.run(qc)

        qc_sched = schedule(qc_opt, backend)

        data["depth"].append(qc_opt.depth())
        data["count_ops"].append(qc_opt.count_ops())
        data["runtime"].append(qc_sched.duration*backend.dt*1e6)

        print(f"{i} depth: {qc_opt.depth()}")
        print(f"{i} counts: {qc_opt.count_ops()}")
        print(f"{i} time: {qc_sched.duration*backend.dt*1e6}")

        print()

with open(outpath, 'rb') as f:
    pkl.dump(data, f)
    

1
Iteration 0
1 depth: 761
1 counts: OrderedDict({'rz': 794, 'sx': 568, 'ecr': 197, 'x': 5})
1 time: 118.86000000000001

Iteration 1
1 depth: 754
1 counts: OrderedDict({'rz': 683, 'sx': 411, 'ecr': 192, 'x': 40})
1 time: 127.56

Iteration 2
1 depth: 680
1 counts: OrderedDict({'rz': 680, 'sx': 494, 'ecr': 160, 'x': 6})
1 time: 106.56000000000002

Iteration 3
1 depth: 689
1 counts: OrderedDict({'rz': 694, 'sx': 505, 'ecr': 160, 'x': 4})
1 time: 106.74000000000001

Iteration 4
1 depth: 794
1 counts: OrderedDict({'rz': 825, 'sx': 589, 'ecr': 201})
1 time: 122.88000000000001

2
Iteration 0
2 depth: 3146
2 counts: OrderedDict({'rz': 3522, 'sx': 2483, 'ecr': 885, 'x': 29})
2 time: 507.24000000000007

Iteration 1
2 depth: 3387
2 counts: OrderedDict({'rz': 3617, 'sx': 2610, 'ecr': 905, 'x': 7})
2 time: 542.7

Iteration 2
2 depth: 3068
2 counts: OrderedDict({'rz': 3132, 'sx': 2288, 'ecr': 739, 'x': 19})
2 time: 481.50000000000006

Iteration 3
2 depth: 3489
2 counts: OrderedDict({'rz': 3321, 'sx'

KeyboardInterrupt: 